In [1]:
# Quickly computes the deviation from the September averages
# for selected cities

In [2]:
import pandas as  pd

In [3]:
# Reading data

In [4]:
sept_history = pd.read_csv("../data/city_september_history.csv")

In [5]:
# Merging

In [84]:
daily_means = pd.read_csv("../data/city_recent_daily_means.csv")

In [85]:
daily_means = daily_means.merge(sept_history[['ID_HDC_G0', 'mean']], on='ID_HDC_G0', suffixes=("", "_sept"))

In [91]:
daily_means[".geo"]

0       {"type":"Polygon","coordinates":[[[-68.6117821...
1       {"type":"Polygon","coordinates":[[[-68.6117821...
2       {"type":"Polygon","coordinates":[[[-68.6117821...
3       {"type":"Polygon","coordinates":[[[-68.6117821...
4       {"type":"Polygon","coordinates":[[[-68.6117821...
                              ...                        
5823    {"type":"Polygon","coordinates":[[[-62.8022467...
5824    {"type":"Polygon","coordinates":[[[-62.8022467...
5825    {"type":"Polygon","coordinates":[[[-62.8022467...
5826    {"type":"Polygon","coordinates":[[[-62.8022467...
5827    {"type":"Polygon","coordinates":[[[-62.8022467...
Name: .geo, Length: 5828, dtype: object

In [8]:
# Cleaning

In [9]:
daily_means = daily_means[["date", "ID_HDC_G0", "UC_NM_MN", "UC_NM_LST",
                           "CTR_MN_NM", "CTR_MN_ISO", 
                           "mean", "mean_sept"]]

In [10]:
daily_means = daily_means.set_index(pd.to_datetime(daily_means.date))

In [11]:
daily_means = daily_means.loc["2024-09-01":]

In [12]:
# Analysis

In [13]:
daily_means['variation_mean'] = daily_means['mean'] / daily_means['mean_sept']

In [14]:
# Format as Datawrapper on selected cities
cities = ['Porto Alegre', 'Florianopolis', 'Curitiba',
          'Sao Paulo', 'Belo Horizonte', 'Rio de Janeiro', 'Vila Velha',
          'Cuiaba', 'Campo Grande', 'Brasilia', 'Goiania',
          'Palmas', 'Manaus', 'Belem', 'Rio Branco', 'Boa Vista', 'Macapa', 'Porto Velho',
          'Sao Luis', 'Teresina', 'Natal', 'Fortaleza', 'Recife', 'Joao Pessoa', 'Aracaju', 'Maceio',
          'Salvador'
         ]

In [15]:
daily_means = daily_means[daily_means.UC_NM_MN.isin(cities)]

In [16]:
def transform_values(df, column):
    """
    Transforms the values in the specified column of the DataFrame.
    
    For values less than 1, it returns 1 divided by the value, and multiplies by -1 to invert the sign.
    For values greater than or equal to 1, it leaves the value unchanged.
    
    Args:
    df: pandas DataFrame
    column: the column where the transformation will be applied
    
    Returns:
    pandas DataFrame with a new column containing the transformed values.
    """
    df['transformed_' + column] = df[column].apply(lambda x: -1 / x if x < 1 else x)
    return df

In [17]:
daily_means = transform_values(daily_means, 'variation_mean').sort_values(by='variation_mean')

In [18]:
# Datawrapper forma
dw_multiline = daily_means.pivot(index='date', columns=['UC_NM_MN'], values='variation_mean')

In [19]:
# Percentage of mean
dw_multiline = (dw_multiline * 100).reset_index()

In [20]:
dw_multiline['baseline'] = 100

In [21]:
dw_multiline.to_csv("../output/datawrapper-multiline.csv", index=False)

In [ ]:
# Table

In [65]:
dw_stripes = (daily_means.pivot(index='UC_NM_MN', columns='date', values='mean') * 1e9).reset_index()

In [66]:
dw_stripes.loc[-1] = ""  # adding a row
dw_stripes.index = dw_stripes.index + 1  # shifting index
dw_stripes = dw_stripes.sort_index()  # sorting by index

In [67]:
dw_stripes["label"] = ""

In [68]:
dw_stripes = dw_stripes[["UC_NM_MN", "label", "2024-09-01", 
                         "2024-09-02", "2024-09-03", "2024-09-04",
                         "2024-09-05", "2024-09-06", "2024-09-07", "2024-09-08",
                         "2024-09-09",]]

In [72]:
dw_stripes.loc[0, 'label'] = '1º de Setembro – 9 de Setembro'
dw_stripes.loc[0, 'UC_NM_MN'] = 'Cidade'

In [73]:
dw_stripes

date,UC_NM_MN,label,2024-09-01,2024-09-02,2024-09-03,2024-09-04,2024-09-05,2024-09-06,2024-09-07,2024-09-08,2024-09-09
0,Cidade,1º de Setembro – 9 de Setembro,,,,,,,,,
1,Aracaju,,5.774878,12.178995,12.456645,10.257351,10.224584,9.72766,7.914113,8.905372,7.934762
2,Belem,,9.29731,9.578344,12.178804,7.348999,7.369967,6.642637,7.876628,9.408293,8.798545
3,Belo Horizonte,,8.321189,17.032488,29.857136,34.032404,30.718047,28.772413,15.861812,21.585042,15.395418
4,Boa Vista,,10.819026,9.894495,9.488356,8.331139,12.441079,9.896644,11.991467,12.005633,10.34759
5,Brasilia,,4.597651,7.168618,8.117225,7.808461,24.568676,25.151049,13.586624,10.974288,10.590771
6,Campo Grande,,18.824624,33.797622,6.123705,7.747383,65.230687,29.08806,28.755817,15.739145,24.726408
7,Cuiaba,,37.136315,33.126155,15.595948,22.578758,48.176884,59.752337,28.985016,17.099894,17.975005
8,Curitiba,,21.066591,24.271822,18.423077,21.333846,32.248418,17.950673,30.237421,37.035728,35.383545
9,Florianopolis,,20.302273,9.750763,10.665398,22.126254,12.407707,8.747425,11.016971,27.540966,17.062724


In [74]:
dw_stripes.to_csv("../output/datawrapper-table.csv", index=False)

In [ ]:
# Map

In [79]:
dw_map = daily_means[['UC_NM_MN', 'CTR_MN_NM', 'mean', 'date']]

In [81]:
dw_map['full_name'] = dw_map['UC_NM_MN'] + ', ' + dw_map['CTR_MN_NM']

/var/folders/yl/bd_xy2c53495c2z2g0w0yrzdtddnbq/T/ipykernel_24697/3517821558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dw_map['full_name'] = dw_map['UC_NM_MN'] + ', ' + dw_map['CTR_MN_NM']


In [94]:
dw_map['mean'] = dw_map['mean'] * 1e9

/var/folders/yl/bd_xy2c53495c2z2g0w0yrzdtddnbq/T/ipykernel_24697/4254382157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dw_map['mean'] = dw_map['mean'] * 1e9


In [100]:
dw_map = dw_map.groupby('full_name')['mean'].mean().reset_index()

In [101]:
dw_map.to_csv("../output/datwrapper-map.csv", index=False)

In [102]:
dw_map

,full_name,mean
0,"Aracaju, Brazil",9.486040
1,"Belem, Brazil",8.722170
2,"Belo Horizonte, Brazil",22.397328
3,"Boa Vista, Brazil",10.579492
4,"Brasilia, Brazil",12.507040
5,"Campo Grande, Brazil",25.559272
6,"Cuiaba, Brazil",31.158479
7,"Curitiba, Brazil",26.439014
8,"Florianopolis, Brazil",15.513387
9,"Fortaleza, Brazil",8.767909
